In [1]:
import nltk
import matplotlib as plt
import os
from nltk.tokenize import RegexpTokenizer
import numpy as np
from sklearn.preprocessing import minmax_scale
from IPython.display import Image
from IPython.core.display import HTML 
import requests
import urllib.request
import time
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from string import punctuation
import pandas as pd

In [2]:
nltk.download('gutenberg')
nltk.download('genesis')
nltk.download('inaugural')
nltk.download('nps_chat')
nltk.download('webtext')
nltk.download('treebank')


[nltk_data] Downloading package gutenberg to
[nltk_data]     C:\Users\julia\AppData\Roaming\nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!
[nltk_data] Downloading package genesis to
[nltk_data]     C:\Users\julia\AppData\Roaming\nltk_data...
[nltk_data]   Package genesis is already up-to-date!
[nltk_data] Downloading package inaugural to
[nltk_data]     C:\Users\julia\AppData\Roaming\nltk_data...
[nltk_data]   Package inaugural is already up-to-date!
[nltk_data] Downloading package nps_chat to
[nltk_data]     C:\Users\julia\AppData\Roaming\nltk_data...
[nltk_data]   Package nps_chat is already up-to-date!
[nltk_data] Downloading package webtext to
[nltk_data]     C:\Users\julia\AppData\Roaming\nltk_data...
[nltk_data]   Package webtext is already up-to-date!
[nltk_data] Downloading package treebank to
[nltk_data]     C:\Users\julia\AppData\Roaming\nltk_data...
[nltk_data]   Package treebank is already up-to-date!


True

In [3]:
from nltk.book import *

*** Introductory Examples for the NLTK Book ***
Loading text1, ..., text9 and sent1, ..., sent9
Type the name of the text or sentence to view it.
Type: 'texts()' or 'sents()' to list the materials.
text1: Moby Dick by Herman Melville 1851
text2: Sense and Sensibility by Jane Austen 1811
text3: The Book of Genesis
text4: Inaugural Address Corpus
text5: Chat Corpus
text6: Monty Python and the Holy Grail
text7: Wall Street Journal
text8: Personals Corpus
text9: The Man Who Was Thursday by G . K . Chesterton 1908


1.In Python, create a method for scoring the vocabulary size of a text, and normalize the score from 0 to 1. It does not matter what method you use for normalization as long as you explain it in a short paragraph. (Various methods will be discussed in the live session.)



In [4]:
#### Method to get the Vocab Size and normalize the score

def n_vocab_size(*arg):
    vocab_size = np.array([])
    vocab_size_norm = np.array([])
    
    #### Getting the Vocab Size
    
    for text in arg:
        print(len(set(text)))
        vocab_size = np.append(vocab_size,len(set(text)))

    #### Normalizing using sklearn preprocessing 
    vocab_size_norm_sklearn = minmax_scale(vocab_size, feature_range=(0,1), axis=0)
    
    return(vocab_size,vocab_size_norm_sklearn)


In [5]:
vocab_size = n_vocab_size(text1,text2,text3,text4,text5,text6,text7,text8,text9)

19317
6833
2789
9913
6066
2166
12408
1108
6807


In [6]:
print("Vocabulary Size", *vocab_size[0],sep='\n* ')
print("Normalized Values by using sklearn", *vocab_size[1],sep='\n* ')

Vocabulary Size
* 19317.0
* 6833.0
* 2789.0
* 9913.0
* 6066.0
* 2166.0
* 12408.0
* 1108.0
* 6807.0
Normalized Values by using sklearn
* 0.9999999999999999
* 0.3144049645779559
* 0.09231698610577188
* 0.48355208962600904
* 0.2722829370091713
* 0.05810313581196112
* 0.6205722444944807
* 0.0
* 0.3129770992366412


Vocabulary score is calculated as the number of distinct words in each text passed in. This will give some numeric value to the variation of words within the given text. While a large vocabulary score will not give us any idea on the complexity of the text, it does mean a larger lexicon will be needed to handle the given text.

Using the example code provided, I am using the `minmax_scale` sklearn method to normalize. This calculates the distance between the minimum and maximum values and scales data to where it falls within that range. It is then transformed between the passed range (min,max) which we are setting to be between 0 and 1. 

2.After consulting section 3.2 in chapter 1 of Bird-Klein, create a method for scoring the long-word vocabulary size of a text, and likewise normalize (and explain) the scoring as in step 1 above.


In [7]:
def long_vocab_size(*arg):
    vocab_size = np.array([])
    vocab_size_norm = np.array([])
    
    #### Getting the Long Word (over 15 letters) Vocab Size
    
    for text in arg:
        vocabulary = set(text)
        long_words = [word for word in vocabulary if len(word) > 15]
        vocab_size = np.append(vocab_size,len(long_words))

    #### Normalizing using sklearn preprocessing 
    vocab_size_norm_sklearn = minmax_scale(vocab_size, feature_range=(0,1), axis=0)
    
    return(vocab_size,vocab_size_norm_sklearn)

In [8]:
long_vocab_sizes = long_vocab_size(text1,text2,text3,text4,text5,text6,text7,text8,text9)

In [9]:
print("Long Vocabulary Size", *long_vocab_sizes[0],sep='\n* ')
print("Normalized Values by using sklearn", *long_vocab_sizes[1],sep='\n* ')

Long Vocabulary Size
* 24.0
* 4.0
* 0.0
* 18.0
* 84.0
* 0.0
* 120.0
* 0.0
* 2.0
Normalized Values by using sklearn
* 0.2
* 0.03333333333333333
* 0.0
* 0.15
* 0.7
* 0.0
* 1.0
* 0.0
* 0.016666666666666666


In [10]:
long_vocab_sizes[0]

array([ 24.,   4.,   0.,  18.,  84.,   0., 120.,   0.,   2.])

Now using a similar method to vocabulary score. We are taking that same word set we simply counted before and are filtering out any words that are not longer than 15 characters. The same normalization technique is being used to get a comparison among the texts. This is starting to give us more metrics around the difficulty of understanding text. We are using the assumption that longer words are more difficult to understand. A work with a larger number of long words should increase the overall difficulty of the text. 

3.Now create a “text difficulty score” by combining the lexical diversity score from homework 1, and your normalized score of vocabulary size and long-word vocabulary size, in equal weighting. Explain what you see when this score is applied to same graded texts you used in homework 1.

In [11]:
#### Method to get the Vocab Size and normalize the score

def vocab_difficulty(*arg):
    vocab_difficulty = np.array([])
    vocab_difficulty_norm = np.array([])
    
    #### Getting the vocab size, normalized
    vocab_sizes = n_vocab_size(*arg)
    normalized_vocab_size = vocab_sizes[1]
    #### Getting the length vocab size, normalized
    long_vocab_sizes = long_vocab_size(*arg)
    normalized_long_vocab_size = long_vocab_sizes[1]
    #### Getting the overall difficulty score (normalized size + normalized length size)
    for i in range(len(normalized_vocab_size)):
        difficulty = normalized_vocab_size[i]*.5 + normalized_long_vocab_size[i]*.5
        vocab_difficulty = np.append(vocab_difficulty,difficulty)

    #### Normalizing using sklearn preprocessing 
    vocab_difficulty_norm_sklearn = minmax_scale(vocab_difficulty, feature_range=(0,1), axis=0)
    
    return(vocab_difficulty,vocab_difficulty_norm_sklearn)

In [12]:
vocab_difficulties = vocab_difficulty(text1,text2,text3,text4,text5,text6,text7,text8,text9)

19317
6833
2789
9913
6066
2166
12408
1108
6807


In [13]:
print("Text Difficulty Score", *vocab_difficulties[0],sep='\n* ')
print("Normalized Values by using sklearn", *vocab_difficulties[1],sep='\n* ')

Text Difficulty Score
* 0.6
* 0.17386914895564462
* 0.04615849305288594
* 0.31677604481300453
* 0.4861414685045856
* 0.02905156790598056
* 0.8102861222472404
* 0.0
* 0.16482188295165393
Normalized Values by using sklearn
* 0.7404791758446576
* 0.21457747353914625
* 0.05696567149005389
* 0.39094344098410655
* 0.5999627232369785
* 0.03585346843335931
* 1.0
* 0.0
* 0.2034119534153422


In [14]:
d = {'Vocab Size Score':vocab_size[0], 'Normalized Vocab Size Score':vocab_size[1], 
    'Long Word Score':long_vocab_sizes[0], 'Normalized Long Word Score':long_vocab_sizes[1],
    'Text difficulty':vocab_difficulties[0],'Normalized Text difficulty':vocab_difficulties[1]}

df = pd.DataFrame(data=d)

In [15]:
df.sort_values(by='Normalized Text difficulty', ascending=False)

,Vocab Size Score,Normalized Vocab Size Score,Long Word Score,Normalized Long Word Score,Text difficulty,Normalized Text difficulty
6,12408.0,0.620572,120.0,1.000000,0.810286,1.000000
0,19317.0,1.000000,24.0,0.200000,0.600000,0.740479
4,6066.0,0.272283,84.0,0.700000,0.486141,0.599963
3,9913.0,0.483552,18.0,0.150000,0.316776,0.390943
1,6833.0,0.314405,4.0,0.033333,0.173869,0.214577
8,6807.0,0.312977,2.0,0.016667,0.164822,0.203412
2,2789.0,0.092317,0.0,0.000000,0.046158,0.056966
5,2166.0,0.058103,0.0,0.000000,0.029052,0.035853
7,1108.0,0.000000,0.0,0.000000,0.000000,0.000000


For the overall difficulty score, we computed the added normalized values of vocabulary size and length. Having no subject matter knowledge on if length or vocabulary size is a 'better' measure of difficulty, we are simply adding the two scores with no weighting. These were then again normalized across the given text input. 

Based on the outcomes, the top two most difficult text had trade offs between having a larger vocabulary size and having more long words. The top overall score had the largest (1.00 Normalized) number of long words, but was also second (0.6 Normalized) in vocabulary size. The second overall score in comparison had the largest vocabulary size (1.00 Normalized), but had a much smaller word length score (0.2 Normalized). The skew in word length left many 3 texts with a score of zero. There may be a benefit of adding in multiple levels of word lengths (maybe small= >5, medium >10, and large>15) to get a better sense of the overall text. 